# GPU computation & batching for speed
The objective of this notebook is to teach you to make training faster from a computational perspective.

1) We will use a GPU to accelerate our computation.

2) We will batch our computation in the sense that the network will process multiple samples **at once**.

Your task: modify the test code to also run in batches on the GPU.

This will allow us to try larger networks :) - play with it.  See what accuracy you can get.

In [ ]:
require 'cutorch'
require 'nn'
require 'cunn'
require 'optim'
print('ready')

In [ ]:
cutorch.setDevice(1)

In [ ]:
x = torch.randn(10,2)
print(x)

In [ ]:
xc = x:cuda()
print(xc)

In [ ]:
-- you cannot operate on vectors living in different memory
-- the errors given are counterintuitive
print(x+xc)

In [ ]:
-- if we create another variable on the GPU, we can happily operate on those
yc = torch.randn(10,2):cuda()
print(torch.mm(yc:t(),xc))

In [ ]:
-- this should be familiar - we load up the data

-- snippet shamelessly ripped from https://github.com/torch/tutorials/blob/master/A_datasets/mnist.lua 
-- load the training data (downloads when first used)
tar = 'http://torch7.s3-website-us-east-1.amazonaws.com/data/mnist.t7.tgz'

if not paths.dirp('mnist.t7') then
   print('==> downloading dataset')
   os.execute('wget ' .. tar)
   os.execute('tar xvf ' .. paths.basename(tar))
end

train_file = 'mnist.t7/train_32x32.t7'
test_file = 'mnist.t7/test_32x32.t7'

----------------------------------------------------------------------
print('==> loading dataset')

-- We load the dataset from disk, it's straightforward

train_data = torch.load(train_file,'ascii')
test_data = torch.load(test_file,'ascii')

In [ ]:
-- let's preprocess the data a bit, so they are between 0 and 1
-- NEW: and move them to the GPU
train_data.data = train_data.data:reshape(60000,1024):type('torch.DoubleTensor'):mul(1.0 / 256.0):cuda()
train_data.labels = train_data.labels:cuda()
test_data.data = test_data.data:reshape(10000,1024):type('torch.DoubleTensor'):mul(1.0 / 256.0):cuda()
test_data.labels = test_data.labels:cuda()

In [ ]:
-- itorch.image will accept arrays living on the GPU
itorch.image(train_data.data[1]:reshape(32,32))

In [ ]:
net = nn.Sequential()
net:add(nn.Linear(1024,700))
net:add(nn.ReLU())
net:add(nn.Linear(700,500))
net:add(nn.ReLU())
net:add(nn.Linear(500,400))
net:add(nn.ReLU())
net:add(nn.Dropout())
net:add(nn.Linear(400, 10))

-- move entire network to the GPU
net = net:cuda()

-- loss is cross-entropy (== logsoftmax + classnllcriterion)
loss = nn.CrossEntropyCriterion():cuda()
print(net)

parameters, gradParameters = net:getParameters()
print('*** Total number of parameters = ' .. parameters:nElement() .. ' ***')

In [ ]:
-- random initialization according to Xavier
local tmp = math.sqrt(1. / net:get(1).bias:size(1))
net:get(1).weight:uniform(-tmp, tmp)
net:get(1).bias:zero()
local tmp = math.sqrt(1. / net:get(3).bias:size(1))
net:get(3).weight:uniform(-tmp, tmp)
net:get(3).bias:zero()
local tmp = math.sqrt(1. / net:get(5).bias:size(1))
net:get(5).weight:uniform(-tmp, tmp)
net:get(5).bias:zero()
local tmp = math.sqrt(1. / net:get(8).bias:size(1))
net:get(8).weight:uniform(-tmp, tmp)
net:get(8).bias:zero()

-- reset the state of the optimizer
opt_state = {}

In [ ]:
execute_optimizer = optim.sgd
opt_config = {
    learningRate = 0.0001,
    momentum = 0.0,
    learingRateDecay = 1e-3
}

--execute_optimizer = optim.adadelta
--opt_config = {
--    rho = 0.9,
--    eps = 1e-6
--}

nepochs = 10
ntrain = 60000
ntest = 10000
batch_size = 100
etest = 1
diagnostic_level = 1

-- train the network
for e=1,nepochs do
    
    local timer = torch.Timer() 
    local confmat = optim.ConfusionMatrix(10, {'0','1','2','3','4','5','6','7','8','9'})
    local train_err = 0
    local inputs = torch.Tensor(batch_size,32*32):cuda()
    local targets = torch.Tensor(batch_size):cuda()
    
    for b=1,ntrain/batch_size do
        
        function batch_eval(x)
            
            local err = 0
            
            if x ~= parameters then
                parameters:copy(x)
            end

            -- fill up the batch vector
            for i=1,batch_size do
                local ndx = (b-1) * batch_size + i
                inputs[i]:copy(train_data.data[ndx])
                targets[i] = train_data.labels[ndx]
            end
            
            gradParameters:zero()
            
            local ys = net:forward(inputs)
            local batch_err = loss:forward(ys, targets)
            local dt_dy = loss:backward(ys, targets)
            net:backward(inputs, dt_dy)
            
            -- add all results into the confusion matrix
            for i=1,batch_size do
                confmat:add(ys[i],targets[i])
            end
        
            train_err = train_err + batch_err
            return batch_err, gradParameters
        end
        
        execute_optimizer(batch_eval, parameters, opt_config, opt_state)
    end
    
    print('******************* EPOCH ' .. e .. ' ************************')

    print('TRAINING [error = ' .. train_err .. ']')
    if diagnostic_level > 0 then
        print(confmat:__tostring__())
    end
    
    -- compute testing error every etest epochs
    if e % etest == 0 then
        local terr = 0
        local misses = 0
        confmat:zero()
        for i=1,ntest do
            local x = test_data.data[i]
            local t = test_data.labels[i]
            local y = net:forward(x)
            confmat:add(y, t)
            local _, digit = torch.max(y,1)
            digit = digit[1]
            if digit ~= t then
                misses = misses + 1
            end
            terr = terr + loss:forward(y, t)
        end
        print('TESTING [error = ' .. terr .. '] misses = ' .. misses)
        if diagnostic_level > 0 then
            print(confmat:__tostring__())
        end
    end
        
    print('Epoch took ' .. timer:time().real .. ' seconds.')
end